In [1]:
def is_igh(obj, igh):
    """Check to see if an object implements a given interface, for example IGH_Component or IGH_Params
        ::return True if the object implements a given interface
    """
    try: 
        if igh(obj):
            return True
    except:
        return False
    
def split_gh_graph_numbering(string):
    return [int(x) for x in string.split("-")]